# UCI Epileptic Seizure Recognition Data Set

Link to the dataset: https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition

In [448]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import transforms

In [449]:
class UCIEpilepsy(Dataset):
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path).iloc[:, 1:].to_numpy().astype(np.single)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):        
        data = torch.from_numpy(self.data[idx,2:])
        label = torch.from_numpy(self.data[idx,178:])
        return data, label

dataset = UCIEpilepsy(data_path='data/data.csv')
batch_size = 64
test_split = .2
shuffle_dataset = True
random_seed = 42
dataset_size = len(dataset)

# Creating data indices for training and validation splits:
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
test_indices, train_indices = indices[:split], indices[split:]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

In [450]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(177, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 5),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.linear_relu_stack(x)

In [451]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=177, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=5, bias=True)
    (5): ReLU()
  )
)


In [463]:
import torch.optim as optim

In [490]:
def train_loop(dataloader, model, criterion, optimizer):
    size = len(dataloader)
    for batch, (X, y) in enumerate(dataloader, 0):
        outputs = model(X)
        loss = criterion(outputs, y.squeeze(1).long()-1)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
               
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}    [{current:>5d}/{len(dataset):>5d}]")

def test_loop(dataloader, model, criterion):
    size = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            outputs = model(X)
            test_loss += criterion(outputs, y.squeeze(1).long()-1)
            correct += (outputs.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss {test_loss:>8f} \n")

In [491]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
epochs = 2

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_dataloader, model, criterion, optimizer)
    test_loop(test_dataloader, model, criterion)
print("Done")

Epoch 1
-------------------------------
loss: 1.609437    [    0/11500]
loss: 1.609437    [  640/11500]
loss: 1.609437    [ 1280/11500]
loss: 1.609437    [ 1920/11500]
loss: 1.609437    [ 2560/11500]
loss: 1.609437    [ 3200/11500]
loss: 1.609437    [ 3840/11500]
loss: 1.609437    [ 4480/11500]
loss: 1.609437    [ 5120/11500]
loss: 1.609437    [ 5760/11500]
loss: 1.609437    [ 6400/11500]
loss: 1.609437    [ 7040/11500]
loss: 1.609437    [ 7680/11500]
loss: 1.609437    [ 8320/11500]
loss: 1.609437    [ 8960/11500]
Test Error: 
 Accuracy: 0.0%, Avg loss 1.609436 

Epoch 2
-------------------------------
loss: 1.609437    [    0/11500]
loss: 1.609437    [  640/11500]
loss: 1.609437    [ 1280/11500]
loss: 1.609437    [ 1920/11500]
loss: 1.609437    [ 2560/11500]
loss: 1.609437    [ 3200/11500]
loss: 1.609437    [ 3840/11500]
loss: 1.609437    [ 4480/11500]
loss: 1.609437    [ 5120/11500]
loss: 1.609437    [ 5760/11500]
loss: 1.609437    [ 6400/11500]
loss: 1.609437    [ 7040/11500]
loss: 